# 1. Lattice attack for ETRU with dimension reduction on the lattice

## 1.1 Environment 

- Needs sagemath installed
- Needs a folder with ETRU private public keys in a pkl format in the folder Chaves/all_keys61.pkl (for n = 61 e.g.). OBS: private key is only used to check if we found the correct key. 
- Folder called Results where the runs of BKZ will be saved
- File with ETRU functions called 'ETRU_Funcoes.sage'


## 1.2 Big picture of how to use these functions

1. Generate ETRU keys using the function gen_key_database on 'ETRU_Funcoes.sage' 
and save in the 'Chaves' folder
2. For each keypair, run run_attack_single_key to construct the appropriated lattice (with a dimension reduction
dictated by the 'cut' parameter). Results are saved automatically in the 'Results' folder with a random name. 
3. Read the results and for each of them, check if the BKZ returned matrix contains the private key or one of its rotations. 


NOTE: We first run results of the BKZ part before checking because this is the most time consumming part 
and only latter we check if it contains the key or not. 

TODO: Ideally, whenever we want to run some BKZ attack for some set of parameters, we should check if this has already been done by reading the files in the Results folder.

# 2. Start of CODE

## 2.1 Import libraries and define functions to 
- Read public key and create the lattice
- Run BKZ on this lattice and save results in a dictionary format (see 'run_attack_single_key')
- Check if correct key or one of its convolutions is in the returned BKZ reduced matrix ('buscar_chave')

In [ ]:
from sage.all import load
import time
import random
import os
import pickle
import numpy as np

load('ETRU_Funcoes.sage')

#Ataque ao NTRU com LLL
def get_alpha(h):
    coefs_linha1 = []
    for i in range(len(h.list())):
        coefs_linha1 += h.list()[i]
    coefs_linha2 =[]
    for j in range(0,len(coefs_linha1),2):
        coefs_linha2 += -coefs_linha1[j+1], coefs_linha1[j]-coefs_linha1[j+1]
    return [coefs_linha1,coefs_linha2]

def rot_alpha(coefs,qtd):
    return [linha[qtd:] + linha[:qtd] for linha in coefs]

def lattice_etru(publickey, q, n):
    """
    Returns the lattice
    """
    coefs = get_alpha(publickey)
    M = matrix(4 * n)
    for i in range(2*n,4*n):
        M[i,i] = q
    for i in range(2*n):
        M[i,i] = 1
    for i in range(n):
        coef_rot = rot_alpha(coefs, 2*i)
        for j, coef in enumerate(coef_rot[0]):
            M[2*i, j+2*n] = coef
        for j, coef in enumerate(coef_rot[1]):
            M[2*i+1, j+2*n] = coef
    return M

    
def run_bkz(M, n, corte = 0, block_size = 10):    
    """
    Run bkz on a lattice
    """    
    M = M[:,:(4*n-corte)] # cut at the end of g
    try:
        m = M.BKZ(block_size = block_size)
    except:
        m = "error in bkz"
    return m


def run_attack_single_key(n, q, h, f, fp, fq, g, block, cut):
    """
    Input 
        n: integer
        q: integer
        h_index: 1 till 100
        block: bkz block in {5,10,15,20,25}
        cut: mays cut on the g part of the lattice, in in range(0,2n-1,5)

    Return
        results (dict): dictionary of the format {'n': n, 'h_index': 3, 
        'bkz': {'block': in {5,10,15,20,25}, 'cut': in range(0,2n-1,5), 'm_reduced': reduced_matrix} }


    # read key at the h_index in pickle format. 
    # keys are with the public key h from 1 till 100 with name chaves_10.pkl where 10 is n

    # create lattice, apply cut, try to run bkz and put result in the dict

    try:
        reduced_matrix = BKZ
    except:
        reduced_matrix = "error in bkz" 

    result = {'n': n, 'h_index': h_index, 
        'bkz': {'block': block, 'cut': cut, 'm_reduced': reduced_matrix} }

    output: something of this format
    {n: value, h_index: 1 till 100, bkz: list_of_dictionaries }
    list_of_dictionaries = {block: in {5,10,15,20,25}, cut: in range(0,2n-1,5), m_reduced: reduced_matrix }
    """

    start_time = time.time()
    
    lattice = lattice_etru(h, q, n)

    reduced_matrix = run_bkz(lattice, n, corte = cut, block_size = block)
    
    total_time = time.time() - start_time

    result = {'n': n, 'q': q, 'f': f, 'fp': fp, 'fq': fq, 'g': g, 'h': h, 'block': block, 'cut': cut, 'm_reduced': reduced_matrix, 'total_time': total_time}

    return result


def get_coefs(h,n):
    coefs = []
    for i in range(len(h.list())):
        coefs += list(reversed(h.list()))[i]
    return [0] * (2*n - len(coefs)) + coefs

def rot_coefs(lista, quantidade):
    quantidade = quantidade % len(lista)  
    return lista[quantidade:] + lista[:quantidade]

def buscar_chave(m_red, f, n):
    submatriz = m_red[:, :2*n]
    coefs_f = get_coefs(f , n)
    chave_encontrada = False
    for i in range(2*n):
        coefs_f_rot = rot_coefs(coefs_f , i)
        for j in range(2*n):
            linha =  submatriz[j].list()
            if linha == coefs_f_rot or linha == -1*coefs_f_rot:
                #print("Chave encontarda")
                #print(f"Rotações: {i}, Linha da matriz {j}")
                chave_encontrada = True
                break
        if chave_encontrada:
            break
    #if not chave_encontrada:
        #print("Chave não encontarda")
    return chave_encontrada


def save_results(res):
    fname = "results/results_" + str(randrange(10^40)) + ".pkl"
    with open(fname,"wb") as f:
        pickle.dump(res,f)
        
        


def merge_pickles(folder_path):
    all_data = []

    for filename in os.listdir(folder_path):
        if filename.endswith('.pkl'):
            with open(os.path.join(folder_path, filename), 'rb') as file:
                data = pickle.load(file)
                all_data.extend(data)

    return all_data

## 2.2 Run bkz on etru keys and save as pickle format

In [ ]:
# run results for several cut_values on specific keys and save
# as pickle format the returned bkz reduced lattice. 
# assumes etru public private key pairs are saved as 
# pkl format in the folder Chaves/all_keys61.pkl (for n = 61 e.g.)

import pickle
total = 100
n_values = [61]
q = 383
cut_values = range(41,60,1)
res = []
block = 20
#all_results = merge_pickles('results')

for n in n_values:
    with open("Chaves/all_keys" + str(n) + ".pkl","rb") as chaves_etru:
        chaves = pickle.load(chaves_etru)
    #total = len(chaves)
    for i in range(total):
        print(i)
        f, fp, h, fq, g,_ = chaves[i]
        for cut in cut_values:
            res.append(run_attack_single_key(n, q, h, f, fp, fq, g, block, cut))
    save_results(res)        


## 2.3 Read bkz runs and find the key for specific n and cut

In [ ]:
# read all data from the results folder containing pkl's in the dictionary format
# as returned by run_attack_single_key
merged_data = merge_pickles('results/n_61')

In [ ]:
merged_data = res

In [ ]:
# compute sucess rate of finding the private key
n = 61
block = 20
print('n', 'cut', 'total', 'worked', 'rate')
for cut in range(120,0,-1):  
    res = [a for a in merged_data if a['n'] == n and a['cut'] == cut and a['block'] == block]
    total = 0
    worked = 0
    for i in range(len(res)):
        candidate  = res[i]
        total += 1
        try:
            found = buscar_chave(candidate['m_reduced'], candidate['f'], candidate['n'])
            
            if found:
                worked += 1

        except:
            1
    if total > 0:
        rate = 1.0 * worked/total
        print(n, cut, total, worked, rate)
    

## 2.4 Time measurement of attack

In [ ]:
# get time taken to run bkz 
times = [a['total_time'] for a in merged_data if a['n'] == 61 and a['cut'] == 49 and a['block'] == 20]
times.sort()
np.mean([t for t in times if t > 40])

In [ ]:
res = [a for a in merged_data if a['n'] == 61]
res
for i in range(len(res)):
    candidate  = res[i]
    try:
        print( i, candidate['n'], candidate['cut'], buscar_chave(candidate['m_reduced'], candidate['f'], candidate['n']))
    except:
        1